In [1]:
# Import modules
import pandas as pd
import numpy as np
import sklearn
import torch
from torch import nn
from sklearn.model_selection import train_test_split
torch.__version__

'2.2.2'

In [2]:
# Load the dataset and review the first 100 characters
with open("the-verdict.txt", "r",encoding="utf-8") as f:
    raw_text = f.read()
print("Length of text:", len(raw_text))
print(raw_text[:9])

Length of text: 20479
I HAD alw


In [3]:
1+1

2

In [4]:
data = [1,2,3]
tens1 = torch.tensor(data)
print(tens1)

tensor([1, 2, 3])


In [5]:
array = np.array([1,2,3])
print(array)
tens2 = torch.from_numpy(array)
print(tens2)

[1 2 3]
tensor([1, 2, 3])


In [21]:
# Ch 2, p 22 - split text
import re
text = "Hello, world!  This, is a test."
result = re.split(r'([!,.]|\s)', text)
print(result)
result2 = [item for item in result if item.strip()]
print(result2)

['Hello', ',', '', ' ', 'world', '!', '', ' ', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']
['Hello', ',', 'world', '!', 'This', ',', 'is', 'a', 'test', '.']


In [25]:
# Add more puncuation and  --- to the split
text = "Hello, world!  This: -- is a test. Maybe?"
result = re.split(r'([!,.?:]|--|\s)', text)
print(result)
result2 = [item for item in result if item.strip()]
print(result2)

['Hello', ',', '', ' ', 'world', '!', '', ' ', '', ' ', 'This', ':', '', ' ', '', '--', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '', ' ', 'Maybe', '?', '']
['Hello', ',', 'world', '!', 'This', ':', '--', 'is', 'a', 'test', '.', 'Maybe', '?']


In [30]:
# p24 - Use tokenizer scheme on Wharton text
preprocessed = re.split(r'([!,.?:;"_()\']|--|\s)', raw_text)
preprocessed = [item for item in preprocessed if item.strip()]
print(len(preprocessed))


4690


In [33]:
# Test output of tokenized used on The Verdict  
print(preprocessed[0:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [34]:
# 2.3 - Create a vocabulary

# Create list of all uniuqe words and sort
all_words   = sorted(set(preprocessed))
vocab_size  = len(all_words)
print(vocab_size)

1130


In [35]:
# Use vocab to create a dictionary
vocab = {token: integer for integer, token in enumerate(all_words)}
for i, item in enumerate(vocab.items()):
    print(item)
    if i > 20:
        break
    

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)


In [44]:
# p 27 - Make simple tokenizer Class
class SimpleTokenV1:
    
    def __init__(self, vocab):
        self.str_to_int = vocab # stores the vocab dictionary
        self.int_to_str = {integer: string for string, integer in vocab.items()} # reverse dictionary (token IDS to text tokens)

    def encode(self, text):
        preprocessed = re.split(r'([!,.?:;"()\']|--|\s)', text)
        preprocessed = [item for item in preprocessed if item.strip()]
        ids = [self.str_to_int[token] for token in preprocessed]
        return ids # return the token IDs

    def decoder(self, ids):  # Converts token IDs back to text
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([!,.?:;"_()\'])', r'\1', text)
        return text
    

In [46]:
# Test the tokenizer
tokenizer = SimpleTokenV1(vocab)
text = """"It's the last he painted, you know," 
Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [47]:
# Test the decoder
print(tokenizer.decoder(ids))

" It' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.


In [50]:
# Need to account for words not in the vocab
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext]>","<[unk]>"])
vocab = {token: integer for integer, token in enumerate(all_tokens)}
print(len(vocab.items()))

1132


In [52]:
# Confirm that new characters are in the vocab
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext]>', 1130)
('<[unk]>', 1131)


In [53]:
# p 31 - Adjust tokenizer class
class SimpleTokenV2:
    
    def __init__(self, vocab):
        self.str_to_int = vocab # stores the vocab dictionary
        self.int_to_str = {integer: string for string, integer in vocab.items()} # reverse dictionary (token IDS to text tokens)

    def encode(self, text):
        preprocessed = re.split(r'([!,.?:;"()\']|--|\s)', text)
        preprocessed = [item for item in preprocessed if item.strip()]

        # Account for words not in the vocab
        preprocessed = [token if token in self.str_to_int else "<[unk]>" for token in preprocessed]

        ids = [self.str_to_int[token] for token in preprocessed]
        return ids # return the token IDs

    def decoder(self, ids):  # Converts token IDs back to text
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([!,.?:;"_()\'])', r'\1', text)
        return text

In [60]:
# Test the tokenizer adjusted for unknown words
test1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext]> ".join((test1, text2))
print(text)

Hello, do you like tea? <|endoftext]> In the sunlit terraces of the palace.


In [62]:
# Get token IDs
tokenizer = SimpleTokenV2(vocab)
print(tokenizer.encode(text))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]


In [63]:
# Test the decoder
print(tokenizer.decoder(tokenizer.encode(text)))

<[unk]>, do you like tea? <|endoftext]> In the sunlit terraces of the <[unk]>.
